### 1.3.1 손실함수
### 1.3.2 미분과 기울기
### 1.3.3 연쇄법칙
### 1.3.4 계산 그래프

In [4]:
# Repeat노드
import numpy as np

D, N = 8, 7
x = np.random.randn(1, D)               # 입력
y = np.repeat(x, N, axis=0)             # 순전파
dy = np.random.randn(N, D)              # 무작위 기울기
dx = np.sum(dy, axis=0, keepdims=True)  # 역전파


- np.repeat()메서드가 원소 복제를 수행
- 배열 x를 N번 복제하는데, 이때 axis를 지정하여 어느축 방향으로 복제할지를 조정할수 있음.
- 역전파에서는 총합을 구해야하므로 np.sum()메스드를 이용. 이때도 axis 인수를 설정하여 어느축 방향으로 합을 구할지 지정.
- 인수로 keepdims=True 를 설정하여 2차원 배열의 차원 수를 유지.
- 이 예에서는 keepdims가 True면 np.sum()의 결과의 형상은 (1,D)가 되며, False면 (D)가 됨.

In [7]:
# Sum노드

D, N = 8, 7
x = np.random.randn(N, D)               # 입력
y = np.sum(x, axis=0, keepdims=True)    # 순전파
dy = np.random.randn(1, D)              # 무작위 기울기
dx = np.repeat(dy, N, axis=0)           # 역전파


- Sum노드의 순전파는 np.sum()메서드로, 역전파는 np.repeat()메서드로 구현.
- Sum노드와 Repeat노드는 서로 '반대관계'


In [8]:
# MatMul노드 (matrix multiply)
class MatMul:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.x = None
    
    def forward(self, x):
        W, = self.params
        out = np.matmul(x,W)
        self.x = x
        return out
    
    def backward(self, dout):
        W, = self.params
        dx = np.matmul(dout, W.T)
        dW = np.matmul(self.x.T, dout)
        self.grads[0][...] = dW
        return dx

- MutMul계층은 학습하는 매개변수를 params에 보관.
- 거기에 대응시키는 형태로, 기울기는 grads에 보관.
- 역전파에서는 dx와 dW를 구해 가중치의 기울기를 인스턴스 변수인 grads에 저장.
- 기울기 값을 설정하는 grads[0][...]=dW 코드에서 점 3개로 이뤄진 생략 기호(...)사용.
- 이렇게 하면 넘파이 배열이 가리키는 메모리 위치를 고정시킨 다음, 그 위치에 원소들을 덮어씀.
- 이처럼 메모리 주소를 고정함으로써 인스턴스 변수 grads를 다루기가 더 쉬워짐.

### 1.3.5 기울기 도출과 역전파 구현

In [9]:
# Sigmoid 계층

class Sigmoid:
    def __init__(self):
        self.params, self.grads = [], []
        self.out = None
        
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        return out
    
    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        return dx

순전파 때는 인스턴스 변수 out에 저장하고, 역전파를 계산할 때 이 out변수를 사용.

In [10]:
# Affine 계층
class Affine:
    def __init__(self, W, b):
        self.params = [W,b]
        self.grads = [np.zeros_like(W), np.zeros_like(b)]
        self.x = None
        
    def forward(self,x):
        W, b = self.params
        out = np.matmul(x,W) + b
        self.x = x
        return out
    
    def backward(self, dout):
        W,b = self.params
        dx = np.matmul(dout, W.T)
        dW = np.matmul(self.x.T, dout)
        db = np.sum(dout, axis=0)
        
        self.grads[0][...] = dW
        self.grads[1][...] = db
        return dx

- 인스턴스 변수 params에는 매개변수를, grads에는 기울기 저장.
- 가중치 매개변수의 기울기를 인스턴스 변수 grads에 저장.

### 1.3.6 가중치 갱신

오차역전파로 기울기를 구했으면, 그 기울기를 사용해 신경망의 매개변수를 갱신함.
- 1단계: 미니배치 _ 훈련 데이터 중에서 무작위로 다수의 데이터를 골라낸다.
- 2단계: 기울기 계산 _ 오차역전파로 각 가중치 매개변수에 대한 손실 함수의 기울기를 구한다.
- 3단계: 매개변수 갱신 _ 기울기를 사용하여 가중치 매개변수를 갱신한다.
- 4단계: 반복 _ 1~3 단계를 필요한 만큼 반복한다.

In [11]:
# SGD
class SGD:
    def __init__(self, lr=0.01):
        self.lr = lr
    
    def update(self, params, grads):
        for i in range(len(params)):
            params[i] -= self.lr * grads[i]

- 초기화 인수 lr은 학습률을 뜻하며, 그 값을 인스턴스 변수로 저장. 그리고 update(params,grads)메서드는 매개변수 갱신을 처리.

In [ ]:
# SGD클래스를 사용하면 신경망의 매개변수 갱신을 다음처럼 할수있음.
# 실제로 동작하지 않는 의사코드
model = TwoLayerNet(...)
optimizer = SGD()

for in range(10000):
    ...
    x_batch, t_batch = get_mini_batch(...)  # 미니배치 획득
    loss = model.forward(x_batch,t_batch)
    model.backward()
    optimizer.update(model.params,model.grads)
    ...
    